# Valid Sudoku

### Difficulty: $\star\star$

### Hint
hashsets, `collections.defaultdict`, //

### Problem
Determine if a 9 x 9 Sudoku board is valid. Only the filled cells need to be validated according to the following rules:
1. Each row must contain the digits 1-9 without repetition.
2. Each column must contain the digits 1-9 without repetition.
3. Each of the nine 3 x 3 sub-boxes of the grid must contain the digits 1-9 without repetition.

## Method 1: Brute Force
Since we are evaluating the validity by **units of 9** (rows/columns/squares), it is natural to think of writing a helper method that evalutes if a unit is valid (if 1-9 appears in this unit at most once).

We create an initially empty list. If a number appears for the first time, we add it to the list. If it appears for the second time (this number already exists in the list), then it breaks the rule and the unit is invalid. 

In [ ]:
def isValid(self, unit):
        l = []
        for num in unit:
            if num != ".":
                if num not in l:
                    l.append(num)
                else:
                    return False
        return True

For the main code, we first evalute the 9 rows. An iteration through all the rows will suffice.

We then evaulate all the columns. We first find the the 9 units by setting the column index constant and iterate through the matrix from top to bottom, and repeat the process 9 times. Then we call `isValid` method to determine validity.

Finally, for each square, we note that each group of row / column indices are sequences of 3: (1,2,3), (4,5,6), (7,8,9). We can store them in a list, and use them as ranges for the row and column indices of each square. Afterwards, we call `inValid` method. 

In [ ]:
def isValidSudoku(self, board):
        """
        :type board: List[List[str]]
        :rtype: bool
        """

        # check rows
        for row in board:
            if not self.isValid(row):
                return False
        
        # check columns
        dic={}
        for i in range(0,9):
            for j in range(0,9):
                if j not in dic:
                    dic[j] = [board[i][j]]
                else:
                    dic[j] += [board[i][j]]
        for unit in dic.values():
            if not self.isValid(unit):
                return False
        
        # check squares
        bound = [[0,1,2], [3,4,5], [6,7,8]]
        for i in bound:
            for j in bound:
                unit = []
                for x in i:
                    for y in j:
                        unit += [board[x][y]]
                if not self.isValid(unit):
                    return False
        
        return True

## Method 2: A Slight Improvement

Method 1 can be cumbersome when we put 9 elements (in a column/square) into a list and then iterate through a list to see if a number occurs more than once. A quick fix will be to to iterate through the elements in a list only once. While we can use a list as we did in method 1, a better option here is a **hash set**.

*Note: To create an empty hash set in python,*
```python
s = set()
s.add(1)
s.add(2)
# s = {1,2}
```
*We use `set()` for construction so as not to confuse this with a dictionary.*

To keep the main method short, we can create a helper method that carries out the duty of using hashset to test the occurances of numbers. 
```python
def isValid(self, s, num):
    if num != ".":
        if num in s:
            return False
        else:
            s.add(num)
```

In [ ]:
def isValidSudoku(self, board):
    # check rows
    for row in board:
        s = set()
        for num in row:
            if self.isValid(s, num) is False:
                return False
    
    # check columns
    for i in range(0,9):
        s = set()
        for j in range(0,9):
            num = board[j][i]
            if self.isValid(s, num) is False:
                return False
    
    # check squares
    bound = [[0,1,2], [3,4,5], [6,7,8]]
    for i in bound:
        for j in bound:
            s = set()
            for x in i:
                for y in j:
                    num = board[x][y]
                    if self.isValid(s, num) is False:
                        return False
    return True

## Method 3: The Elegant Way

This method is much more elegant than previous ones in two ways.

1. The method iterates through every element in the sudoku only once. The idea is, for every element, it can only exist in its respective row, column, and square only once. So we are performing **three** checks at the same time. This will require us to store more than one hash set at any given time (in fact, we will have 9*3=27 hash sets). 
    
    To create the hash sets, we import `defaultdict` class from the `collections` module. It creates an instance of `defaultdict` by calling `collections.defaultdict(set)`. `defaultdict` is just like a dictionary except that it provides a default value for missing keys,simplifying certain operations. Here, the values of `defaultdict` is made up of hash sets.

2. The method greatly simplifies the process of attributing elements to its respective squares. The idea here is that the board can be broken down into nine small squares, and each square can by charactized by row and column index of 0, 1, 2. And to determine the row and column index of the square any element belongs to, we only need to perform a `//` operation to find the dividend. 



In [ ]:
def isValidSudoku(self, board):
       
        columns = collections.defaultdict(set)
        rows = collections.defaultdict(set)
        squares = collections.defaultdict(set)

        for r in range(0, 9):
            for c in range(0, 9):
                # skip if empty
                if board[r][c] == ".":
                    continue
                if (board[r][c] in rows[r]
                    or board[r][c] in columns[c]
                    or board[r][c] in squares[(r//3, c//3)]):
                    return False
                # if not duplicate, add to hashsets
                rows[r].add(board[r][c])
                columns[c].add(board[r][c])
                squares[(r//3, c//3)].add(board[r][c])
        return True